In [1]:
# !pip install transformers datasets torch
# !pip install transformers[torch]
#!pip uninstall numpy
#!pip install numpy==1.26.4
#!pip install --upgrade transformers
#!pip install sentencepiece

## Import Libs

In [2]:
import os
import pandas as pd
import torch

from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelWithLMHead, Trainer, TrainingArguments
from transformers import LlamaTokenizer, LlamaForCausalLM, TrainingArguments, Trainer
from datasets import Dataset

from transformers import LlamaTokenizer, LlamaForCausalLM


c:\Users\cibel\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
print(torch.__version__)


2.5.0+cu124


In [4]:
print(torch.cuda.is_available())  # Deve retornar True
print(torch.cuda.get_device_name(0))  # Nome da sua GPU

True
NVIDIA GeForce RTX 4060


In [5]:
# Verificar se a GPU está disponível
if torch.cuda.is_available():
    device = torch.device("cuda")  # Define a GPU como dispositivo
    print(f"Usando GPU: {torch.cuda.get_device_name(0)}")  # Imprime o nome da GPU
else:
    device = torch.device("cpu")  # Se não houver GPU disponível, usa a CPU
    print("Usando CPU")

# Exemplo de como mover um tensor para a GPU
tensor = torch.randn(3, 3)  # Tensor aleatório
tensor = tensor.to(device)  # Mover para o dispositivo correto (GPU ou CPU)
print(tensor)

Usando GPU: NVIDIA GeForce RTX 4060
tensor([[-1.5453,  0.2253,  0.6028],
        [-0.3446, -1.7721,  0.7371],
        [-0.2583, -0.8747, -0.9884]], device='cuda:0')


In [6]:
import subprocess

# Verificar o uso da GPU
subprocess.run(["nvidia-smi"])

CompletedProcess(args=['nvidia-smi'], returncode=0)

## Import Data

In [7]:
caminho_redacoes_corrigidas = os.path.join('..', 'data', 'redacoes_corrigidas_uol_fora_enem.csv')

df_redacoes_corrigidas = pd.read_csv(caminho_redacoes_corrigidas, sep=";")
df_redacoes_corrigidas.head()

,tema,description,title,text,comments,criteria_score_1,criteria_score_2,criteria_score_3,criteria_score_4,criteria_score_5
0,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,Natureza caótica,A natureza possui um equilíbrio singular singu...,"Texto muito bom, com alguns erros e equívocos ...",160,200,160,200,160
1,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,Em busca de um novo lar,"Com o aumento do aquecimento global, várias es...",Texto mediano. Além dos problemas pontuais neg...,120,120,120,120,120
2,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,A ação antrópica e suas consequências,"Há, na pré-história, pinturas rupestres que re...","Texto prolixo e repetitivo, com problemas pont...",120,120,80,120,120
3,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,O aquecimento global e os ursos,O aquecimento global vem afetando o mundo nos ...,"Lamentavelmente, o texto é fraco, insuficiente...",120,80,80,80,80
4,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,As consequências do degelo no Ártico,"O aumento do aquecimento global provocou, no í...",Texto mediano. O autor cumpre as exigências da...,80,120,120,120,120


In [8]:
df_redacoes_corrigidas["nota_final"] = df_redacoes_corrigidas["criteria_score_1"] + df_redacoes_corrigidas["criteria_score_2"] + df_redacoes_corrigidas["criteria_score_3"] + df_redacoes_corrigidas["criteria_score_4"] + df_redacoes_corrigidas["criteria_score_5"]

In [9]:
df_redacoes_corrigidas['tema'][0]

'Os ursos polares da Rússia e um dilema ecológico'

In [10]:

# Passo 2: Pré-processar os dados
# Unir a redação e correção no formato de entrada do modelo

prompt_input = """
    Você é um estudante do ensino médio que está prestando vestibular para o ENEM, o Exame Nacional do Ensino Médio. 
    Você deve se atentar à 
     - Domínio da norma-padrão da língua escrita; 
     - Compreensão do tema; 
     - Organização das informações e argumentos;
     - Correta aplicação da lógica;
    - Apresentar uma proposta de intervenção para o problema.
    
    O tema da redaçãao neste ano é "{}". 
    E você escreveu a seguinte redação:

    {}
""".format(df_redacoes_corrigidas['tema'][0],  df_redacoes_corrigidas['text'][0])

print(prompt_input)


    Você é um estudante do ensino médio que está prestando vestibular para o ENEM, o Exame Nacional do Ensino Médio. 
    Você deve se atentar à 
     - Domínio da norma-padrão da língua escrita; 
     - Compreensão do tema; 
     - Organização das informações e argumentos;
     - Correta aplicação da lógica;
    - Apresentar uma proposta de intervenção para o problema.
    
    O tema da redaçãao neste ano é "Os ursos polares da Rússia e um dilema ecológico". 
    E você escreveu a seguinte redação:

    A natureza possui um equilíbrio singular singular, de modo que todos os seres vivos desempenham uma função - os ursos polares, por exemplo, ao se alimentarem de peixes e focas focas, regulam a quantidade de indivíduos nessas populações. Contudo, os homens ainda não compreendem a importância do respeito ao meio ambiente para a preservação da própria espécie e, por isso, têm arcado com as consequências, como o ataque dos ursos polares a comunidades humanas por diminuição da oferta de a

In [11]:
prompt_output = """
   A correção da sua redação foi a seguinte:
   {}. 
""".format(df_redacoes_corrigidas['comments'][0])

print(prompt_output)


   A correção da sua redação foi a seguinte:
   Texto muito bom, com alguns erros e equívocos em termos de conteúdo, que impedem a atribuição de uma nota ainda melhor. Parágrafo 1) Texto muito bom em termos de linguagem. Os erros gramaticais apontados em verde impedem que se lhe atribua a nota máxima. Destaque-se que o título não tem nada que ver com o texto. Parágrafo 2) O autor compreendeu o tema e desenvolveu um texto dissertativo-argumentativo obedecendo as exigências desta competência. Parágrafo 3) Argumentação boa. O autor contextualiza bem o problema. Só se equivoca ao dizer que os homens não compreendem a necessidade de respeitar o meio ambiente. A afirmação é excessivamente genérica e contraditória, pois, se vários acordos e protocolos foram assinados desde os anos 70, uma parte significativa das nações se mostra preocupada com o problema. Também é estranho argumentar com o "segundo maior poluidor". Seria melhor apontar o primeiro maior ou os dois países. Parágrafo 4) A argum

In [12]:
df_input_output = pd.DataFrame()

In [13]:
df_input_output['input'] = df_redacoes_corrigidas.apply(
    lambda x: prompt_input.format(x['tema'], x['text']), axis=1
)
df_input_output['output'] = df_redacoes_corrigidas.apply(
    lambda x: prompt_output.format(x['comments']), axis=1
)

train_data = Dataset.from_pandas(df_input_output[['input', 'output']])
train_data

Dataset({
    features: ['input', 'output'],
    num_rows: 3726
})

In [14]:

# tokenizer = LlamaTokenizer.from_pretrained("maritaca-ai/sabia-7b")
# model = LlamaForCausalLM.from_pretrained(
#     "maritaca-ai/sabia-7b",
#     device_map="auto",  # Automatically loads the model in the GPU, if there is one. Requires pip install acelerate
#     low_cpu_mem_usage=True,
#     torch_dtype=torch.bfloat16   # If your GPU does not support bfloat16, change to torch.float16
# )  

# prompt = """Classifique a resenha de filme como "positiva" ou "negativa".

# Resenha: Gostei muito do filme, é o melhor do ano!
# Classe: positiva

# Resenha: O filme deixa muito a desejar.
# Classe: negativa

# Resenha: Apesar de longo, valeu o ingresso.
# Classe:"""

# input_ids = tokenizer(prompt, return_tensors="pt")

# output = model.generate(
#     input_ids["input_ids"].to("cuda"),
#     max_length=1024,
#     eos_token_id=tokenizer.encode("\n"))  # Stop generation when a "\n" token is dectected

# # The output contains the input tokens, so we have to skip them.
# output = output[0][len(input_ids["input_ids"][0]):]

# print(tokenizer.decode(output, skip_special_tokens=True))

In [15]:
# Carregando o tokenizer e o modelo pré-treinado
tokenizer = LlamaTokenizer.from_pretrained("wandgibaut/periquito-3B")
tokenizer.pad_token = tokenizer.eos_token

model = LlamaForCausalLM.from_pretrained(
    "wandgibaut/periquito-3B",
    device_map="auto",
    torch_dtype=torch.bfloat16,  # Troca para bfloat16
    low_cpu_mem_usage=True
).to("cuda")

Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.16s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
You shouldn't move a model that is dispatched using accelerate hooks.


RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.

In [23]:
# tokenizer = AutoTokenizer.from_pretrained("Giuliano/llama-2-7b-periquito-pt-br")
# model = AutoModelForCausalLM.from_pretrained("Giuliano/llama-2-7b-periquito-pt-br",
#     device_map="auto",
#     offload_folder="./offload",  # Move partes do modelo para o disco
#     low_cpu_mem_usage=True,
#     torch_dtype=torch.float16)

In [24]:


# # Carregando o tokenizer e o modelo pré-treinado
# tokenizer = LlamaTokenizer.from_pretrained("maritaca-ai/sabia-2-tokenizer-small")
# tokenizer.pad_token = tokenizer.eos_token

# model = LlamaForCausalLM.from_pretrained(
#     "maritaca-ai/sabia-2-tokenizer-small",
#     device_map="auto",
#     offload_folder="./offload",  # Move partes do modelo para o disco
#     low_cpu_mem_usage=True,
#     torch_dtype=torch.float16
# )

In [19]:
# Criando o dataset compatível com Hugging Face
train_data = Dataset.from_pandas(df_input_output[['input', 'output']])

# Tokenizando os dados
def preprocess_data(batch):
    inputs = tokenizer(batch['input'], max_length=512, truncation=True, padding="max_length")
    outputs = tokenizer(batch['output'], max_length=512, truncation=True, padding="max_length")

    # Para o modelo de causal LM, deslocamos os labels
    labels = outputs['input_ids']
    inputs['labels'] = labels
    return inputs

tokenized_data = train_data.map(preprocess_data, batched=True, remove_columns=["input", "output"])

# **Treinando o modelo**
# Configuração de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no", #"epoch"
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=10_000,
    save_total_limit=2,
    push_to_hub=False,  # Defina como True se desejar fazer upload para o Hugging Face Hub
    fp16=True  # Se a GPU suportar
)

# Criando o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    tokenizer=tokenizer,
)

Map: 100%|██████████| 3726/3726 [00:09<00:00, 401.45 examples/s]
c:\Users\cibel\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\cibel\AppData\Local\Temp\ipykernel_19904\1586421316.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
# Iniciando o treinamento
trainer.train()


  0%|          | 0/5589 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 21.22 GiB is allocated by PyTorch, and 1.30 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Salvando o modelo treinado
model.save_pretrained("./trained_model_maritaca_v1")
tokenizer.save_pretrained("./trained_model_maritaca_v1")

## Carregar o modelo e o tokenizer ajustados

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM

def testar_modelo(prompt, modelo_caminho="./trained_model", max_length=512, device="cuda"):
    """
    Método para testar o modelo treinado.
    
    Parâmetros:
        - prompt (str): O texto de entrada para o modelo.
        - modelo_caminho (str): Caminho para o modelo treinado salvo.
        - max_length (int): Comprimento máximo da geração.
        - device (str): Dispositivo a ser usado ('cuda' para GPU, 'cpu' para CPU).
    
    Retorno:
        - output_text (str): O texto gerado pelo modelo.
    """
    # Carregar o modelo treinado e o tokenizer
    tokenizer = LlamaTokenizer.from_pretrained(modelo_caminho)
    model = LlamaForCausalLM.from_pretrained(modelo_caminho)
    model.to(device)

    # Tokenizar o prompt
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)

    # Gerar texto com o modelo
    output_ids = model.generate(
        input_ids=input_ids["input_ids"],
        max_length=max_length,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        num_return_sequences=1,  # Gerar apenas uma saída
        temperature=0.7,         # Controlar aleatoriedade
        top_k=50                 # Limitar escolhas de palavras
    )

    # Decodificar o texto gerado
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    # Retornar apenas a parte gerada
    return output_text[len(prompt):].strip()

# Exemplo de uso
prompt_teste = """Tema: Tecnologia na educação
Redação: A tecnologia tem transformado a maneira como aprendemos. Desde o uso de computadores em sala de aula até ferramentas como inteligência artificial, os avanços tecnológicos têm ampliado o acesso à educação. Contudo, é necessário que haja um equilíbrio entre inovação e inclusão digital.

Correção:"""

resultado = testar_modelo(prompt_teste)
print("Saída do modelo:", resultado)


In [23]:
# Carregar o modelo e o tokenizer ajustados
model = AutoModelForCausalLM.from_pretrained("./modelo_finetuned_enem_v3")
tokenizer = AutoTokenizer.from_pretrained("./modelo_finetuned_enem_v3")

# Definir o dispositivo (GPU se disponível)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [24]:
print(torch.cuda.is_available())

True


In [25]:
device

device(type='cuda')

In [34]:
# # Função para avaliar redação
# def avaliar_redacao(redacao, tema, max_length = 750):
#     """
#     Avalia uma redação baseada em um tema usando um modelo fine-tuned GPT-like.

#     Args:
#         redacao (str): O texto da redação.
#         tema (str): O tema da redação.
#         max_length (int): Comprimento máximo da resposta gerada.

#     Returns:
#         str: Resposta gerada pelo modelo.
#     """
#     # Criar o prompt com a redação e o tema
#     prompt_input = f"""
#     Você é um estudante do ensino médio que está prestando vestibular para o ENEM, o Exame Nacional do Ensino Médio.
#     Você deve se atentar à:
#     - Domínio da norma-padrão da língua escrita;
#     - Compreensão do tema;
#     - Organização das informações e argumentos;
#     - Correta aplicação da lógica;
#     - Apresentar uma proposta de intervenção para o problema.

#     O tema da redação neste ano é "{tema}".
#     E você escreveu a seguinte redação:

#     {redacao}
#     """
    
#     # Tokenizar o prompt usando o tokenizer e mover os tensores para o dispositivo correto
#     inputs = tokenizer(
#         prompt_input,
#         return_tensors="pt",  # Gera tensores PyTorch
#         truncation=True # Limita o comprimento do prompt
#     )
#     input_ids = inputs["input_ids"].to(device)
#     attention_mask = inputs["attention_mask"].to(device)
    
#     # Gerar a saída do modelo
#     outputs = model.generate(
#         input_ids=input_ids,  # IDs tokenizados
#         attention_mask=attention_mask,  # Máscara de atenção
#         max_length=max_length,  # Comprimento máximo do texto gerado
#         num_return_sequences=1,  # Gera apenas uma sequência
#         do_sample=True,  # Sampling para maior diversidade
#         top_p=0.95,  # Nucleus sampling
#         top_k=50  # Limita o número de tokens considerados na geração
#     )
    
#     # Decodificar a resposta do modelo
#     resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
#     return resposta

In [39]:
def avaliar_redacao(redacao, tema, max_length = 750):
    # Criar o prompt com a redação e o tema
    
    prompt_input_ = """
        Você é um estudante do ensino médio que está prestando vestibular para o ENEM, o Exame Nacional do Ensino Médio. 
        Você deve se atentar à 
        - Domínio da norma-padrão da língua escrita; 
        - Compreensão do tema; 
        - Organização das informações e argumentos;
        - Correta aplicação da lógica;
        - Apresentar uma proposta de intervenção para o problema.
        
        O tema da redaçãao neste ano é "{}". 
        E você escreveu a seguinte redação:

        {}
    """.format(tema, redacao)
    
    prompt_input = "Oi, tudo bem?"

    # Tokenizar o prompt
    inputs = tokenizer(prompt_input, 
                            return_tensors="pt"
                            ).to(device)
    #inputs = tokenizer(prompt, return_tensors="pt").to(device) - # PT é de temnsorflow = TODO: Usar modelos, maritaca, sabias, caramelo
    
    # Gerar a saída do modelo
    outputs = model.generate(inputs["input_ids"], max_length=max_length, num_return_sequences=1, do_sample=True, top_p=0.95, top_k=50)
    
    # Decodificar a resposta do modelo
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return resposta

In [40]:
# Exemplo de uso
redacao_exemplo = """
A imigração de refugiado tem sido um dos maiores desafios das nações. Este transpassa a questão humanitária e vai 
de encontro a  soberania de muitos países. Não diferente, O  Brasil enfrenta, como os demais  situação parecida. 
Nos últimos anos uma grande quantidade de refugiados têm transpassada  a fronteira do país forçando o poder 
público a tomar medidas que garantam a integridade da soberania nacional e que esteja em consonância aos direitos humanos.  
Várias nações do mundo estão passando por percalços, seja por  guerras, crises econômicas, conflitos internos, regimes ditatoriais... 
Fato que  faz surgir refugiados que vislumbram nos países mais próximos a salva guarda  para manterem a integridade. O que para estes  
é um alento, para o país em que vão se refugiam  se torna um problema. Que muitas vezes vezes,  não está relacionado diretamente 
a um sentimento xenófobo, mas sim por  uma questão social. No caso do Brasil, os refugiados da nossa nação adjacente se abrigaram 
em uma cidade de um estado da federação  que tem grande dificuldade para cuidar até mesmo dos nativos. Causando  um verdadeiro caos. 
O que  fez aumentar ainda mais o sofrimento de quem já está em uma situação extrema. Destarte que  não basta o mero acolhimento, 
deve-se socorrer e fomentar uma nova expectativa de vida nestas  pessoas que estão distantes do país de origem. O que  tende a 
demandar tempo e recursos. No caso brasileiro  um dispêndio a mais em um país que está em busca de reduzir despesas. Salientando 
que se faz necessário uma previa  triagem para evitar que pessoas criminosas e mal-intencionadas venham fazer da nação um local 
para praticas criminosa  e potencialize a violência. Desprende  que o maior desafio no trato com os imigrantes está em conciliar 
a soberana força nacional de proteção com a aplicação de direitos humanos. É impensável deixar de ajudar o semelhante em apuros. 
Efetivar esse apoio sem prejudicar os demais é tarefa que requer recursos e planejamentos. O que aos poucos o país vai se consolidando.

"""
tema_exemplo = "Desafios na imigração de refugiados"

correcao = avaliar_redacao(redacao_exemplo, tema_exemplo)
print(correcao)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Oi, tudo bem? se*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?)*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".*teen?".**teen?)*teen?".*teen?".teen?".*teen?".teen?".*teen?".teen?".*teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".teen?".t